<a href="https://colab.research.google.com/github/nancyandrioli/aa-en-grafos/blob/main/exploratorioViz_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! pip install pandas --upgrade
# !pip install --upgrade pandas==1.2

     |████████████████████████████████| 9.9MB 5.4MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.0 which is incompatible.
  Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

print(pd.__version__)

1.2.0


In [ ]:
os.getcwd()

'/content'

In [ ]:
! curl -L 'https://github.com/vast-challenge/2021-sample-data/blob/main/MC3.zip?raw=true' -o MC3.zip
! echo 'n' | unzip MC3.zip
csv_files_list = ["MC3/csv-1700-1830.csv", "MC3/csv-1831-2000.csv", "MC3/csv-2001-2131.csv"]
csv_files_list

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   133  100   133    0     0    517      0 --:--:-- --:--:-- --:--:--   517
100   144  100   144    0     0    342      0 --:--:-- --:--:-- --:--:--   342
100  862k  100  862k    0     0  1157k      0 --:--:-- --:--:-- --:--:-- 1157k
Archive:  MC3.zip
replace MC3/10 year historical document.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace MC3/AnswerSheet/~$ST Challenge 2021 MC3 Answer Sheet.htm? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


['MC3/csv-1700-1830.csv', 'MC3/csv-1831-2000.csv', 'MC3/csv-2001-2131.csv']

In [ ]:
csv_files_list

['MC3/csv-1700-1830.csv', 'MC3/csv-1831-2000.csv', 'MC3/csv-2001-2131.csv']

In [ ]:
df_csv_files = (pd.read_csv(f, encoding='ISO-8859-1') for f in csv_files_list)
df  = pd.concat(df_csv_files, ignore_index=True)

In [ ]:
df.head(3)

,type,date(yyyyMMddHHmmss),author,message,latitude,longitude,location
0,mbdata,20140123170000,POK,Follow us @POK-Kronos,NaN,NaN,NaN
1,mbdata,20140123170000,maha_Homeland,Don't miss a moment! Follow our live coverage of the POK Rally in the Park!,NaN,NaN,NaN
2,mbdata,20140123170000,Viktor-E,Come join us in the Park! Music tonight at Abila City Park!,NaN,NaN,NaN


In [ ]:
df['date'] = pd.to_datetime(df['date(yyyyMMddHHmmss)'], format='%Y%m%d%H%M%S')

In [ ]:
# quito los segundos agrupar a nivel minuto
df['dateYMDHM'] = pd.to_datetime(df['date(yyyyMMddHHmmss)'].astype(str).str.slice(0,12), format='%Y%m%d%H%M')

In [ ]:
df.groupby(['type']).date.count()

type
ccdata     191
mbdata    3872
Name: date, dtype: int64

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop = stopwords.words('english')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
df.loc[df['type'] == 'ccdata','message_clean']  = (df.loc[df['type'] == 'ccdata','message'].str.replace('[^a-zA-Z0-9]+',' ').str.lower()
                                                                                    .apply(word_tokenize)
                                                                                     .apply(lambda x: [item for item in x if item.isalpha()])
                                                                                     .apply(lambda x: [item for item in x if item not in stop])
                                                                                     .apply(' '.join))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.


In [ ]:
df.loc[df['type'] == 'mbdata','message_clean']  = (df.loc[df['type'] == 'mbdata','message'].str.replace('[^a-zA-Z0-9]+',' ').str.lower()
                                                                                    .apply(word_tokenize)
                                                                                     .apply(lambda x: [item for item in x if item.isalpha()])
                                                                                     .apply(lambda x: [item for item in x if item not in stop])
                                                                                     .apply(' '.join))

In [ ]:
df[df['type']=='ccdata'].head(3)

,type,date(yyyyMMddHHmmss),author,message,latitude,longitude,location,date,dateYMDHM,message_clean,hashtags,mentions,rt,message_topic,message_topics,message_max_topic,message_max_topic_perc,message_clean_hhm
9,ccdata,20140123170000,NaN,KEEP THE PEACE-CROWD CONTROL/ABILA CITY PARK,NaN,NaN,Egeou St / Parla St,2014-01-23 17:00:00,2014-01-23 17:00:00,keep peace crowd control abila city park 170,[],[],False,[],[],NaN,NaN,keep peace crowd control abila city park 170
28,ccdata,20140123170210,NaN,TRAFFIC STOP,NaN,NaN,N. Els St / N. Polvo St,2014-01-23 17:02:10,2014-01-23 17:02:00,traffic stop 170,[],[],False,[],[],NaN,NaN,traffic stop 170
59,ccdata,20140123170513,NaN,PARK CHECK,NaN,NaN,2099 Sannan Pky,2014-01-23 17:05:13,2014-01-23 17:05:00,park check 170,[],[],False,[],[],NaN,NaN,park check 170


In [ ]:
# columna de hashtags, @ y rt
df['hashtags'] = df['message'].str.findall(r'#.*?(?=\s|$)')
df['mentions'] = df['message'].str.findall(r'@.*?(?=\s|$)')
df['rt'] = df['message'].str.startswith('RT ')

In [ ]:
df[df['type']=='ccdata'].groupby(['message']).type.count().sort_values(ascending=False)

message
TRAFFIC STOP                                                                        36
ALARM-SECURE NO CRIME                                                               15
INCOMPLETE CALL FOR POLICE                                                          15
SUBJECT STOP                                                                        13
DISTURBANCE-NOISE                                                                   12
SUSPICIOUS SUBJECT/CIRCUMSTANCES-IN PROGRESS                                        11
SUSPICIOUS OCCUPIED VEHICLE-BLACK VAN                                               11
MISDEMEANOR ASSAULT-IN PROGRESS                                                     11
VEHICLE ACCIDENT-REPORT                                                              8
PARK CHECK                                                                           5
CHECK ON HAZARD                                                                      5
BUSINESS CHECK                     

In [ ]:
%%capture
!pip install pyLDAvis

In [ ]:
from collections import Counter
from nltk import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords

import gensim
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from gensim import corpora

import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

import os, re, operator, warnings
warnings.filterwarnings('ignore')  
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
remove_users = ['KronosQuoth','Clevvah4Evah','POK']

df[(df['type']=='mbdata') 
    & (~df['message'].str.startswith('RT ')) 
    & (~df['author'].isin(remove_users))
    ].groupby(['author']).author.count().nlargest(20)

author
megaMan                  106
AbilaPost                 72
HomelandIlluminations     65
truccotrucco              64
FriendsOfKronos           63
choconibbs                50
KronosStar                49
roger_roger               37
protoGuy                  34
CentralBulletin           30
GreyCatCollectibles       30
Viktor-E                  30
SaveOurWildlands          26
maha_Homeland             25
panopticon                25
trollingsnark             25
ourcountryourrights       23
prettyRain                23
NewsOnlineToday           21
anaregents                20
Name: author, dtype: int64

In [ ]:
remove_users = ['KronosQuoth','Clevvah4Evah','POK']
del_msjs = ['creditratings','contest','refill','prescription','morecredit','clickhere','pictures','recommend','badprofiles','lucio','speakers','click','cheap car sales','\$','link','diet','stuff','workathome','song']

df[(df['type']=='mbdata')&(~df['author'].isin(remove_users))&(~df['message_clean'].str.contains('|'.join(del_msjs)))&(df['message'].str.lower().str.contains('choconibbs'))].head(5)

,type,date(yyyyMMddHHmmss),author,message,latitude,longitude,location,date,dateYMDHM,message_clean,hashtags,mentions,rt,message_topic,message_topics,message_max_topic,message_max_topic_perc,message_clean_hhm
119,mbdata,20140123171100,rnbwBrite,RT @choconibbs Lovely words - ugly person #POKRally,NaN,NaN,NaN,2014-01-23 17:11:00,2014-01-23 17:11:00,rt choconibbs lovely words ugly person pokrally,[#POKRally],[@choconibbs],True,"[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]","[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]",[0],[0.33333334],rt choconibbs lovely words ugly person pokrally 171
219,mbdata,20140123171701,dragonRider1,"RT @choconibbs POK trying to give an aura of cred - while kidnapping, extorting",NaN,NaN,NaN,2014-01-23 17:17:01,2014-01-23 17:17:00,rt choconibbs pok trying give aura cred kidnapping extorting,[],[@choconibbs],True,"[(0, 0.1668235), (1, 0.18311836), (2, 0.65005815)]","[(0, 0.16682361), (1, 0.18324006), (2, 0.6499364)]",[2],[0.6499364],rt choconibbs pok trying give aura cred kidnapping extorting 171
304,mbdata,20140123172300,lordWally,RT @choconibbs IT just doesn't get better than this #blather #POKRally,NaN,NaN,NaN,2014-01-23 17:23:00,2014-01-23 17:23:00,rt choconibbs get better blather pokrally,"[#blather, #POKRally]",[@choconibbs],True,"[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]","[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]",[0],[0.33333334],rt choconibbs get better blather pokrally 172
375,mbdata,20140123172651,plasticParts,RT @choconibbs @TomaToma kidnapping is a crime #fact #POKRally,NaN,NaN,NaN,2014-01-23 17:26:51,2014-01-23 17:26:00,rt choconibbs tomatoma kidnapping crime fact pokrally,"[#fact, #POKRally]","[@choconibbs, @TomaToma]",True,"[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]","[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]",[0],[0.33333334],rt choconibbs tomatoma kidnapping crime fact pokrally 172
391,mbdata,20140123172725,gardener4958,RT @choconibbs @Sara people first; environment second,NaN,NaN,NaN,2014-01-23 17:27:25,2014-01-23 17:27:00,rt choconibbs sara people first environment second,[],"[@choconibbs, @Sara]",True,"[(0, 0.11892591), (1, 0.7652345), (2, 0.11583966)]","[(0, 0.11893514), (1, 0.7652238), (2, 0.11584105)]",[1],[0.7652238],rt choconibbs sara people first environment second 172


In [ ]:
# df['message_clean'] = df['message_clean'] + ' ' + df['date(yyyyMMddHHmmss)'].astype(str).str.slice(start=8, stop=11)

In [ ]:
del_words = ['abila','get','hi','pok','kronos','viktor','pokrally','followers','rally','sylvia','newman','around','abilapost','many','says','centralbulletin']

desde = df['dateYMDHM'].dt.strftime('%H:%M').min()
hasta = df['dateYMDHM'].dt.strftime('%H:%M').max()

msjs = df[(df['type']=='mbdata') 
    & (~df['message'].str.startswith('RT ')) 
    & (~df['author'].isin(remove_users))
    & (~df['message_clean'].str.contains('|'.join(del_msjs)))
    & ((df['date'].dt.strftime('%H:%M') > desde) & (df['date'].dt.strftime('%H:%M') < hasta))
    ]['message_clean'].apply(word_tokenize).apply(lambda x: [word for word in x if word not in del_words and len(word)>2])

In [ ]:
# msjs

34        [concerns, violence, surround, today, heavy, police, presence]
47      [today, city, park, concerns, violence, heavy, police, presence]
48                           [marek, terrorist, introducing, terrorists]
49                           [long, terrorists, start, destroying, park]
57                                           [meet, band, show, tonight]
                                      ...                               
4044                         [unknown, explosion, heard, dancingdolphin]
4045                          [ohnoohnoohnoohnoohnoohnoohnoohnoohnoohno]
4048                     [explosion, heard, dancing, dolphin, apartment]
4049                                                [didnt, sound, good]
4050                                                [may, creator, help]
Name: message_clean, Length: 1206, dtype: object

In [ ]:
# msjs_expl = msjs.explode()
# msjs_expl.groupby(msjs_expl).count().sort_values(ascending=False)

message_clean
fire              143
police             99
tag                89
apd                80
van                77
                 ... 
dophin              1
dorel               1
downwithkronos      1
obviously           1
zawahiri            1
Name: message_clean, Length: 2079, dtype: int64

In [ ]:
dictionary = corpora.Dictionary(msjs)
dictionary.filter_extremes(no_below=20, no_above=0.5)
print(len(dictionary))

46


In [ ]:
corpus = [dictionary.doc2bow(text) for text in msjs]
ldamodel = LdaModel(corpus=corpus, num_topics=8, id2word=dictionary)
# ldamodel.print_topics(10, 10)
pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.248205 -0.001140       1        1  23.514298
6     -0.076913 -0.085212       2        1  14.432851
1      0.101945 -0.106583       3        1  12.908181
3      0.064123 -0.099081       4        1  11.118792
4     -0.015308  0.107954       5        1  10.640274
7      0.157367  0.052785       6        1  10.075690
5     -0.028562  0.030384       7        1   8.962085
0      0.045552  0.100894       8        1   8.347830, topic_info=        Term        Freq       Total Category  logprob  loglift
25      fire  128.000000  128.000000  Default  30.0000  30.0000
12      cops   66.000000   66.000000  Default  29.0000  29.0000
9     people   71.000000   71.000000  Default  28.0000  28.0000
42       tag   92.000000   92.000000  Default  27.0000  27.0000
43  standoff   57.000000   57.000000  Default  26.0000  26.0000
..       ...         ...         ...      ...      ...      ...
0     police    1.943344   96.250725   Topic8  -4.3579  -1.4194
4        apd    1.030754   79.271018   Topic8  -4.9920  -1.8594
32       cop    1.028174   42.703069   Topic8  -4.9945  -1.2433
26   dancing    1.027009   44.868643   Topic8  -4.9957  -1.2939
28   dolphin    1.026084   44.027133   Topic8  -4.9966  -1.2759

[295 rows x 6 columns], token_table=      Topic      Freq Term
term                      
24        1  0.341541  afd
24        2  0.248394  afd
24        6  0.093148  afd
24        7  0.186295  afd
24        8  0.124197  afd
...     ...       ...  ...
40        4  0.117466  van
40        5  0.065259  van
40        6  0.065259  van
40        7  0.039155  van
40        8  0.052207  van

[280 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 7, 2, 4, 5, 8, 6, 1])

In [ ]:
ldamodel.print_topics(10, 10)

[(0,
  '0.124*"fire" + 0.051*"police" + 0.045*"kronosstar" + 0.042*"dancing" + 0.041*"dolphin" + 0.030*"afd" + 0.022*"dancingdolphinfire" + 0.021*"residents" + 0.019*"scene" + 0.018*"department"'),
 (1,
  '0.081*"tag" + 0.071*"van" + 0.059*"people" + 0.052*"cops" + 0.031*"guy" + 0.030*"still" + 0.029*"think" + 0.027*"cop" + 0.026*"one" + 0.023*"got"'),
 (2,
  '0.061*"standoff" + 0.052*"apd" + 0.039*"gelatogalore" + 0.036*"shooting" + 0.035*"like" + 0.034*"going" + 0.032*"terrorists" + 0.026*"swat" + 0.026*"police" + 0.024*"getting"')]

In [ ]:
bow_vector = dictionary.doc2bow(word_tokenize('panopticon man woman surrended police two people left van think hostages'))
lda_vector = ldamodel[bow_vector]
print(lda_vector)
print(ldamodel.print_topic(max(lda_vector, key=lambda item: item[1])[0]))

[(0, 0.053543177), (1, 0.8914502), (2, 0.055006593)]
0.081*"tag" + 0.071*"van" + 0.059*"people" + 0.052*"cops" + 0.031*"guy" + 0.030*"still" + 0.029*"think" + 0.027*"cop" + 0.026*"one" + 0.023*"got"


In [ ]:
ldatopics = [[word for word, prob in topic] for topicid, topic in ldamodel.show_topics(25,formatted=False)]

In [ ]:
lda_coherence = CoherenceModel(topics=ldatopics, texts=msjs, dictionary=dictionary, window_size=30,coherence='c_npmi',topn=5).get_coherence_per_topic()

In [ ]:
df_lda = pd.DataFrame([range(25),lda_coherence,ldatopics],index=['topic_id','coherence','topic']).transpose()

In [ ]:
df_lda.sort_values(by='coherence',ascending=False)[:3]

,topic_id,coherence,topic
0,0,0.161076,"[fire, police, kronosstar, dancing, dolphin, afd, dancingdolphinfire, residents, scene, department]"
2,2,0.022674,"[standoff, apd, gelatogalore, shooting, like, going, terrorists, swat, police, getting]"
1,1,0.021021,"[tag, van, people, cops, guy, still, think, cop, one, got]"


In [ ]:
df.head(3)

,type,date(yyyyMMddHHmmss),author,message,latitude,longitude,location,date,dateYMDHM,message_clean,hashtags,mentions,rt,message_topic
0,mbdata,20140123170000,POK,Follow us @POK-Kronos,NaN,NaN,NaN,2014-01-23 17:00:00,2014-01-23 17:00:00,follow us pok kronos,[],[@POK-Kronos],False,"[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]"
1,mbdata,20140123170000,maha_Homeland,Don't miss a moment! Follow our live coverage of the POK Rally in the Park!,NaN,NaN,NaN,2014-01-23 17:00:00,2014-01-23 17:00:00,miss moment follow live coverage pok rally park,[],[],False,"[(0, 0.16673759), (1, 0.16671804), (2, 0.6665444)]"
2,mbdata,20140123170000,Viktor-E,Come join us in the Park! Music tonight at Abila City Park!,NaN,NaN,NaN,2014-01-23 17:00:00,2014-01-23 17:00:00,come join us park music tonight abila city park,[],[],False,"[(0, 0.066705845), (1, 0.066790394), (2, 0.8665037)]"


In [ ]:
df['message_topics'] = np.empty((len(df), 0)).tolist()
df.loc[(df['type']=='mbdata'),['message_topics']] = ldamodel[df.loc[(df['type']=='mbdata'),'message_clean'].apply(word_tokenize).apply(dictionary.doc2bow)]

In [ ]:
from operator import itemgetter
df.loc[df['type'] == 'mbdata','message_max_topic']= df.loc[df['type'] == 'mbdata','message_topics'].apply(lambda t: [max(t, key=itemgetter(1))[0]])
df.loc[df['type'] == 'mbdata','message_max_topic_perc']= df.loc[df['type'] == 'mbdata','message_topics'].apply(lambda t: [max(t, key=itemgetter(1))[1]])

In [ ]:
df.head(1)

,type,date(yyyyMMddHHmmss),author,message,latitude,longitude,location,date,dateYMDHM,message_clean,hashtags,mentions,rt,message_topic,message_topics,message_max_topic,message_max_topic_perc
0,mbdata,20140123170000,POK,Follow us @POK-Kronos,NaN,NaN,NaN,2014-01-23 17:00:00,2014-01-23 17:00:00,follow us pok kronos,[],[@POK-Kronos],False,"[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]","[(0, 0.33333334), (1, 0.33333334), (2, 0.33333334)]",[0],[0.33333334]


In [ ]:
df.to_csv('df_topics.csv')